In [1]:
import pandas as pd
import numpy as np
import cv2
import tqdm
import glob
import sys, os
import gc

In [2]:
ROOT_DIR = 'E:\ML\Datasets\Celeb-DF-v2'
FAKE_DIR = 'Celeb-synthesis'
REAL_DIR = ['Celeb-real', 'YouTube-real']

In [3]:
fake_vids_all = os.listdir(os.path.join(ROOT_DIR,FAKE_DIR))
real_vids_all = []

for _ in REAL_DIR:
    real_vids_all.extend(os.listdir(os.path.join(ROOT_DIR, _)))

print(len(fake_vids_all), " : ", fake_vids_all[0])
print(len(real_vids_all), " : " , real_vids_all[0])

5639 :id0_id16_0000.mp4
890 :id0_0000.mp4


In [4]:
_TEST_FILE_LIST = open(r"E:\\ML\\Datasets\\Celeb-DF-v2\\List_of_testing_videos.txt","r").readlines()

TEST_LIST = []

for _ in _TEST_FILE_LIST:
    _label = _.strip().split()[0]
    _id = _.strip().split()[1]
    TEST_LIST.append((_id,_label))

print(len(TEST_LIST))
TEST_LIST[:5]

518


[('YouTube-real/00170.mp4', '1'),
 ('YouTube-real/00208.mp4', '1'),
 ('YouTube-real/00063.mp4', '1'),
 ('YouTube-real/00024.mp4', '1'),
 ('YouTube-real/00021.mp4', '1')]

In [5]:
TRAIN_LIST = []

_FLIST = [x[0].split('/')[1] for x in TEST_LIST]
for _ in fake_vids_all:
    if _ not in _FLIST:
        TRAIN_LIST.append((_,0))
for _ in real_vids_all:
    if _ not in _FLIST:
        TRAIN_LIST.append((_,0))

print(len(TRAIN_LIST))
TRAIN_LIST[:5]

6011


[('id0_id16_0000.mp4', 0),
 ('id0_id16_0001.mp4', 0),
 ('id0_id16_0002.mp4', 0),
 ('id0_id16_0005.mp4', 0),
 ('id0_id16_0006.mp4', 0)]

In [13]:
import tensorflow as tf
import tensorflow.keras as keras

from tensorflow.keras.applications import DenseNet201
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Dense,GlobalAveragePooling2D,Convolution2D,BatchNormalization
from tensorflow.keras.layers import Flatten,MaxPooling2D,Dropout

In [9]:
FRAME_SIZE = 224

In [15]:
dense_net = DenseNet201(
    weights = 'imagenet',
    include_top = False,
    input_shape = (FRAME_SIZE, FRAME_SIZE, 3)
)

x = dense_net.output
x = GlobalAveragePooling2D()(x)
x = BatchNormalization()(x)
x = Dropout(0.5)(x)
x = Dense(1024,activation='relu')(x) 
x = Dense(512,activation='relu')(x) 
x = BatchNormalization()(x)
x = Dropout(0.5)(x)
prediction = Dense(1,activation='sigmoid')(x)

model = Model(inputs = dense_net.input, outputs = prediction)
model.summary()

________________________________
conv5_block20_1_bn (BatchNormal (None, 7, 7, 128)    512         conv5_block20_1_conv[0][0]
__________________________________________________________________________________________________
conv5_block20_1_relu (Activatio (None, 7, 7, 128)    0           conv5_block20_1_bn[0][0]
__________________________________________________________________________________________________
conv5_block20_2_conv (Conv2D)   (None, 7, 7, 32)     36864       conv5_block20_1_relu[0][0]
__________________________________________________________________________________________________
conv5_block20_concat (Concatena (None, 7, 7, 1536)   0           conv5_block19_concat[0][0]
                                                                 conv5_block20_2_conv[0][0]
__________________________________________________________________________________________________
conv5_block21_0_bn (BatchNormal (None, 7, 7, 1536)   6144        conv5_block20_concat[0][0]
_____________________